In [1]:
from transformers import Swinv2Config, Swinv2Model, AutoFeatureExtractor
import transformers
import os, json, glob, requests, torch, cv2
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing import image
from transformers import Swinv2ForImageClassification

In [2]:
#set default path
TRAIN_JSON_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\label\train'
VALID_JSON_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\label\val'
TEST_JSON_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\label\test'

TRAIN_IMG_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\train'
VALID_IMG_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\val'
TEST_IMG_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\test'

TRAIN_SEGMENT_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\segmentation\train'
VALID_SEGMENT_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\segmentation\val'

MASKED_TRAIN_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\masked_train'
MASKED_VAL_PATH = r'C:\Users\User\Desktop\Code\Github\Final_project\data\masked_val'



IMG_ROWS = 256
IMG_COLS = 256

In [3]:
def get_json_list_1d(json_folder_path):
    temp_loader = []
    json_train = []
    emotion_list_json = os.listdir(json_folder_path)

    for emotion in emotion_list_json:
        with open (json_folder_path + '\\' + emotion, "r", encoding='euc-kr') as f:
            temp_loader.append(json.load(f))

    for i in range(len(temp_loader)):
        json_train += temp_loader[i]

    return json_train

In [4]:
def get_json_list_1d(json_folder_path): #returns a 1 dim array
    temp_loader = []
    json_train = []
    emotion_list_json = os.listdir(json_folder_path)

    for emotion in emotion_list_json:
        if 'neutral' not in emotion:
            with open (json_folder_path + "\\" + emotion, "r", encoding='euc-kr') as f:
                temp_loader.append(json.load(f))     
        else:
            with open(json_folder_path + "\\" + emotion, "r", encoding='utf-8') as f:
                temp_loader.append(json.load(f)) 

    for i in range(len(temp_loader)):
        json_train += temp_loader[i]     

    return json_train

In [5]:
#call for the functions
json_train = get_json_list_1d(TRAIN_JSON_PATH)
json_val = get_json_list_1d(VALID_JSON_PATH)




print(json_train[0].keys()) #see what keys are available
print(len(json_train))
print(len(json_val))
print(os.listdir(TRAIN_IMG_PATH))

#5 emotion
#0 = 분노, 1 = 기쁨, 2 = 중립, 3 = 당황, 4 = 슬픔


#4 emotion
#0 = 분노, 1 = 기쁨, 2 = 당황, 3 = 슬픔
#0 = anger, 1 = happy, 2 = panic, 3 = sadness


dict_keys(['filename', 'gender', 'age', 'isProf', 'faceExp_uploader', 'bg_uploader', 'annot_A', 'annot_B', 'annot_C'])
6780
1461
['anger', 'happy', 'neutral', 'panic', 'sadness']


In [6]:
start_point , end_point = np.array_split(list(json_train[0]['annot_A']['boxes'].values()), 2)
print(start_point)
print(end_point)

[2073.254605 1486.204442]
[1338.220105  539.506642]


In [7]:
df = pd.read_csv(r'C:\Users\User\Desktop\Code\Github\Final_project\data\filesize2.csv')

In [8]:
def load_image_and_label_1D(json_list, img_path):
    img_file = []
    label_list = []
    
    # 5 emotions
    kr_to_num = { '분노'   : '0',
                   '기쁨'   : '1',
                   '중립'   : '2',
                   '당황'   : '3',
                   '슬픔'   : '4'             
                 }
    kr_to_en = { '분노'    : 'anger',
                 '기쁨'   : 'happy',
                 '중립'    : 'neutral',
                 '당황'   : 'panic',
                 '슬픔'   : 'sadness'             
                 }
    

    
    for entry in range(len(json_list)):
        try:
            label = json_list[entry]['faceExp_uploader']
            img_dir = img_path + '/' + kr_to_en[label] + '/' + json_list[entry]['filename']

            if label != '중립' :
                start_point , end_point = np.array_split(list(json_list[entry]['annot_A']['boxes'].values()), 2)
                img = cv2.imread(img_dir)  # Load the full image
                roi = img[int(end_point[1]):int(start_point[1]), int(end_point[0]):int(start_point[0])] 
                roi = cv2.resize(roi, (IMG_ROWS, IMG_COLS))
                im_pil = Image.fromarray(roi)
                
            else:
                start_point , end_point = np.array_split(list(json_list[entry]['annot_A']['boxes'].values()), 2)
                height = df.loc[df['file'] == json_list[entry]['filename'], 'height'].values[0]
                length = df.loc[df['file'] == json_list[entry]['filename'], 'width'].values[0]
                h_ratio = 640 / height
                l_ratio = 480 / length
                start_point[0] = start_point[0]*l_ratio
                start_point[1]  = start_point[1]*h_ratio
                end_point[0]    = end_point[0]*l_ratio
                end_point[1]    = end_point[1]*h_ratio
                img = cv2.imread(img_dir)  # Load the full image
                roi = img[int(end_point[1]):int(start_point[1]), int(end_point[0]):int(start_point[0])] 
                roi = cv2.resize(roi, (IMG_ROWS, IMG_COLS))
                im_pil = Image.fromarray(roi)
                

                
            # img_dir = img_path + kr_to_en[label] + '/' + json_list[entry]['filename']
            # img = np.array(image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS)))/255
            # img = image.load_img(img_dir, target_size=(IMG_ROWS, IMG_COLS))
            img_file.append(im_pil)
            label_list.append(int(kr_to_num[label]))
        except Exception:
            print("Emotion " + label + " " + json_list[entry]['filename'] + " Not available")
            pass

    return img_file, label_list

In [9]:
train_img, train_label = load_image_and_label_1D(json_train, TRAIN_IMG_PATH)

Emotion 분노 br810daccfda5c78d9119189c2767db0d19290f2a6166af257307a1fa7b6bflyz.jpg Not available
Emotion 기쁨 341zcc976a81d9d8391dbdc967042a4977d9c594b924adacd11d0b3b0b611z22y.jpg Not available
Emotion 중립 7d316d614dbd22b33df44768199cffa8212ad676673c6bf02b50e1834f8cf2cb2020210210134642010001.jpg Not available
Emotion 중립 7d316d614dbd22b33df44768199cffa8212ad676673c6bf02b50e1834f8cf2cb2020210210134417004002.jpg Not available
Emotion 당황 twdva8d723865fa1005873a6967797c75f3dd83197f13f7daa3e3926947071al6.jpg Not available
Emotion 당황 5yid6d7ae32bf14b1343da46cfdb3fdffc8e1c64baf4fb1b175690f648a870v69.jpg Not available
Emotion 당황 bpv531e5cebdb1c00085158bb0677c5596ccabf1c8a5cb0007991571bf306wcd4.jpg Not available
Emotion 당황 s1q397a38900c1e1f950496109900e7c01f0630459790d8590010655c4090vlh0.jpg Not available
Emotion 당황 4hym2e606d89fcf887b29e7b8b3ebfcc626c8fcf821782f6e5ffee5416c75qzrf.jpg Not available
Emotion 당황 7m2k46c05803855399ea1ee1911875ee59e05640c93135c0bc91b5660456cdo8k.jpg Not available
Emotion 

In [10]:
val_img, val_label = load_image_and_label_1D(json_val, VALID_IMG_PATH)

Emotion 중립 bd210c67f8f6d1d3b34e1f724f26a0460a5e683af5c500b8acacc8be9af35db02020210210223903003009.jpg Not available


In [11]:
from transformers import Swinv2ForImageClassification, AutoFeatureExtractor
model_name = "microsoft/swinv2-tiny-patch4-window8-256"
train_df = pd.DataFrame(columns=['image', 'label'])
train_df['image'] = train_img
train_df['label'] = train_label

val_df = pd.DataFrame(columns=['image', 'label'])
val_df['image'] = val_img
val_df['label'] = val_label



In [12]:
train_dict = train_df.to_dict('records')
val_dict = val_df.to_dict('records')

In [13]:
from datasets import Dataset
# train_ds = Dataset.from_pandas(pd.DataFrame(data=train_dict))
train_ds = Dataset.from_list(train_dict)
val_ds = Dataset.from_list(val_dict)

In [14]:
train_ds[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x256>,
 'label': 0}

In [15]:
# 5 emotions
label2id = {       'Anger'     : '0',
                   'Happy'     : '1',
                   'Neutral'   :'2',
                   'Panic'     : '3',
                   'Sadness'   : '4'             
                 }
id2label = {
    '0' : 'Anger',
    '1' : 'Happy',
    '2' : 'Neutral',
    '3' : 'Panic',
    '4' : 'Sadness'
}

In [16]:
from transformers import AutoImageProcessor

image_processor  = AutoImageProcessor.from_pretrained(model_name)
image_processor

c:\Users\User\anaconda3\envs\my_env_py311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


ViTImageProcessor {
  "_valid_processor_keys": [
    "images",
    "do_resize",
    "size",
    "resample",
    "do_rescale",
    "rescale_factor",
    "do_normalize",
    "image_mean",
    "image_std",
    "return_tensors",
    "data_format",
    "input_data_format"
  ],
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 256,
    "width": 256
  }
}

In [17]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomRotation,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "height" in image_processor.size:
    size = (image_processor.size["height"], image_processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            RandomRotation(degrees=(0.0, 30.0)),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [train_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [18]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)


In [19]:
model = Swinv2ForImageClassification.from_pretrained(
    model_name, 
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-tiny-patch4-window8-256 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments, Trainer

batch_size = 8

model_checkpoint = model_name.split("/")[-1]

args = TrainingArguments(
    f"{model_checkpoint}-finetuned-5emotions",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=35,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    # push_to_hub=True,
)

In [21]:
import numpy as np

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}



In [22]:
from huggingface_hub import login

login()

In [23]:
from datasets import load_metric

metric = load_metric("accuracy")

C:\Users\User\AppData\Local\Temp\ipykernel_51540\1780215247.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
c:\Users\User\anaconda3\envs\my_env_py311\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [25]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

  0%|          | 0/7385 [00:00<?, ?it/s]

{'loss': 1.691, 'grad_norm': 7.075885772705078, 'learning_rate': 6.765899864682003e-07, 'epoch': 0.05}
{'loss': 1.6712, 'grad_norm': 9.890098571777344, 'learning_rate': 1.3531799729364006e-06, 'epoch': 0.09}
{'loss': 1.6787, 'grad_norm': 7.415971755981445, 'learning_rate': 2.029769959404601e-06, 'epoch': 0.14}
{'loss': 1.6685, 'grad_norm': 6.3063764572143555, 'learning_rate': 2.7063599458728013e-06, 'epoch': 0.19}
{'loss': 1.6143, 'grad_norm': 8.763900756835938, 'learning_rate': 3.3829499323410016e-06, 'epoch': 0.24}
{'loss': 1.5849, 'grad_norm': 6.661700248718262, 'learning_rate': 4.059539918809202e-06, 'epoch': 0.28}
{'loss': 1.5502, 'grad_norm': 7.178672790527344, 'learning_rate': 4.736129905277402e-06, 'epoch': 0.33}
{'loss': 1.5274, 'grad_norm': 5.857969284057617, 'learning_rate': 5.4127198917456026e-06, 'epoch': 0.38}
{'loss': 1.4915, 'grad_norm': 6.652136325836182, 'learning_rate': 6.089309878213803e-06, 'epoch': 0.43}
{'loss': 1.4063, 'grad_norm': 6.222568035125732, 'learning_r

  0%|          | 0/183 [00:00<?, ?it/s]

{'eval_loss': 0.8002648949623108, 'eval_accuracy': 0.6835616438356165, 'eval_runtime': 6.3464, 'eval_samples_per_second': 230.051, 'eval_steps_per_second': 28.835, 'epoch': 1.0}
{'loss': 0.9431, 'grad_norm': 14.492610931396484, 'learning_rate': 1.4884979702300405e-05, 'epoch': 1.04}
{'loss': 0.8959, 'grad_norm': 11.852544784545898, 'learning_rate': 1.5561569688768607e-05, 'epoch': 1.09}
{'loss': 0.9688, 'grad_norm': 11.708285331726074, 'learning_rate': 1.6238159675236808e-05, 'epoch': 1.14}
{'loss': 0.8888, 'grad_norm': 14.45132827758789, 'learning_rate': 1.6914749661705008e-05, 'epoch': 1.18}
{'loss': 0.846, 'grad_norm': 14.281059265136719, 'learning_rate': 1.759133964817321e-05, 'epoch': 1.23}
{'loss': 0.8906, 'grad_norm': 14.860888481140137, 'learning_rate': 1.826792963464141e-05, 'epoch': 1.28}
{'loss': 0.8169, 'grad_norm': 10.605212211608887, 'learning_rate': 1.894451962110961e-05, 'epoch': 1.33}
{'loss': 0.836, 'grad_norm': 12.668191909790039, 'learning_rate': 1.962110960757781e-

  0%|          | 0/183 [00:00<?, ?it/s]

...

{'eval_loss': 0.41630348563194275, 'eval_accuracy': 0.9089041095890411, 'eval_runtime': 6.2634, 'eval_samples_per_second': 233.101, 'eval_steps_per_second': 29.217, 'epoch': 33.0}
{'loss': 0.1724, 'grad_norm': 4.449179649353027, 'learning_rate': 3.0469455311465546e-06, 'epoch': 33.04}
{'loss': 0.2433, 'grad_norm': 11.228163719177246, 'learning_rate': 2.9717123081552815e-06, 'epoch': 33.09}
{'loss': 0.2095, 'grad_norm': 9.91794204711914, 'learning_rate': 2.896479085164009e-06, 'epoch': 33.14}
{'loss': 0.1858, 'grad_norm': 11.325150489807129, 'learning_rate': 2.8212458621727358e-06, 'epoch': 33.18}
{'loss': 0.1808, 'grad_norm': 11.480091094970703, 'learning_rate': 2.7460126391814627e-06, 'epoch': 33.23}
{'loss': 0.1749, 'grad_norm': 6.75496244430542, 'learning_rate': 2.6707794161901896e-06, 'epoch': 33.28}
{'loss': 0.1445, 'grad_norm': 7.288575649261475, 'learning_rate': 2.5955461931989165e-06, 'epoch': 33.33}
{'loss': 0.1576, 'grad_norm': 17.545753479003906, 'learning_rate': 2.520312970

  0%|          | 0/183 [00:00<?, ?it/s]

{'eval_loss': 0.420358806848526, 'eval_accuracy': 0.9075342465753424, 'eval_runtime': 6.1196, 'eval_samples_per_second': 238.577, 'eval_steps_per_second': 29.904, 'epoch': 34.0}
{'loss': 0.137, 'grad_norm': 7.621824741363525, 'learning_rate': 1.4670478483298226e-06, 'epoch': 34.04}
{'loss': 0.1626, 'grad_norm': 4.038236141204834, 'learning_rate': 1.3918146253385495e-06, 'epoch': 34.08}
{'loss': 0.1651, 'grad_norm': 5.0053558349609375, 'learning_rate': 1.3165814023472766e-06, 'epoch': 34.13}
{'loss': 0.1458, 'grad_norm': 12.053594589233398, 'learning_rate': 1.2413481793560035e-06, 'epoch': 34.18}
{'loss': 0.1848, 'grad_norm': 8.431727409362793, 'learning_rate': 1.1661149563647307e-06, 'epoch': 34.22}
{'loss': 0.1581, 'grad_norm': 6.528326034545898, 'learning_rate': 1.0908817333734578e-06, 'epoch': 34.27}
{'loss': 0.1662, 'grad_norm': 8.678977012634277, 'learning_rate': 1.015648510382185e-06, 'epoch': 34.32}
{'loss': 0.1751, 'grad_norm': 4.814255714416504, 'learning_rate': 9.404152873909

  0%|          | 0/183 [00:00<?, ?it/s]

{'eval_loss': 0.4179632365703583, 'eval_accuracy': 0.9061643835616439, 'eval_runtime': 6.2707, 'eval_samples_per_second': 232.83, 'eval_steps_per_second': 29.183, 'epoch': 34.96}
{'train_runtime': 2527.3996, 'train_samples_per_second': 93.531, 'train_steps_per_second': 2.922, 'train_loss': 0.334529117377287, 'epoch': 34.96}
***** train metrics *****
  epoch                    =      34.9586
  total_flos               = 7154902524GF
  train_loss               =       0.3345
  train_runtime            =   0:42:07.39
  train_samples_per_second =       93.531
  train_steps_per_second   =        2.922


In [27]:
metrics = trainer.evaluate()
# some nice to haves:
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

  0%|          | 0/183 [00:00<?, ?it/s]

***** eval metrics *****
  epoch                   =    34.9586
  eval_accuracy           =      0.911
  eval_loss               =     0.3806
  eval_runtime            = 0:00:06.06
  eval_samples_per_second =    240.766
  eval_steps_per_second   =     30.178


In [28]:
from transformers import AutoModelForImageClassification, AutoImageProcessor

# pretrain_path = r"C:\Users\User\Desktop\Code\Github\Final_project\WASSUP_EST_FINAL_Team4\swinv2-tiny-patch4-window8-256-finetuned-emotions-train-masked\checkpoint-2516"
# pretrain_path = r'C:\Users\User\Desktop\Code\Github\Final_project\swinv2-tiny-patch4-window8-256-finetuned-eurosat\checkpoint-2516'
pretrain_path = r'C:\Users\User\Desktop\Code\Github\Final_project\WASSUP_EST_FINAL_Team4\swinv2-tiny-patch4-window8-256-finetuned-5emotions\checkpoint-5281'
model_trained = AutoModelForImageClassification.from_pretrained(pretrain_path)
test_img_processor = AutoImageProcessor.from_pretrained(pretrain_path)

In [29]:
json_test = get_json_list_1d(TEST_JSON_PATH)
test_img, test_label = load_image_and_label_1D(json_test, TEST_IMG_PATH)

Emotion 분노 stwqfff0a1f114d3475946c1d4b321d5df08776b1d626373dcdfe242bbad4haix.jpg Not available
Emotion 중립 18682daa07165675e5fe8ee4b8985cc46d6cb509c83e5e5944b257b4c63492d33020210119175208008006.jpg Not available
Emotion 당황 8bj909c3084349a3804abb06e89e69b3305a51b298beb50639d51dd020389jqpq.jpg Not available


In [30]:
predictions = []

for test in test_img:
    enc = test_img_processor(test.convert("RGB"), return_tensors="pt")
    
    with torch.no_grad():
        outputs = model_trained(**enc)
        logits = outputs.logits
        predicted_class_idx = logits.argmax(-1).item()
        predictions.append(model_trained.config.id2label[predicted_class_idx])

predictions
# int_to_label_mapping = {0: '분노', 1: '기쁨', 2: '당황', 3: '슬픔'}
# predicted_labels = [int_to_label_mapping[pred] for pred in predictions]

['Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Panic',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Panic',
 'Sadness',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Panic',
 'Anger',
 'Anger',
 'Anger',
 'Sadness',
 'Sadness',
 'Anger',
 'Sadness',
 'Anger',
 'Panic',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Sadness',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Sadness',
 'Anger',
 'Panic',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Panic',
 'Sadness',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Sadness',
 'Sadness',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Sadness',
 'Sadness',
 'Anger',
 'Panic',
 'Anger',
 'Sadness',
 'Anger',
 'Anger',
 'Anger',
 'Panic',
 'Anger',
 'Happy',
 'Anger',
 'Anger',
 'Anger',
 'Sadness',
 'Anger',
 'Panic',
 'Anger',
 'Panic',
 'Sadness',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 'Anger',
 '

In [31]:
# int_to_label_mapping = {0: '분노', 1: '기쁨', 2: '당황', 3: '슬픔'}
int_to_label_mapping = {0: 'Anger', 1: 'Happy', 2: 'Neutral', 3: 'Panic', 4: 'Sadness'}
test_label = list(map(int_to_label_mapping.get, test_label))

In [34]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
labels = ['Anger', 'Happy', 'Neutral', 'Panic', 'Sadness']

accuracy = accuracy_score(test_label,predictions)
print('Accuracy: {:.2f}'.format(accuracy*100))

report = classification_report(test_label,predictions)
print(report)

cm = confusion_matrix(test_label,predictions)
print(cm)

Accuracy: 89.95
              precision    recall  f1-score   support

       Anger       0.83      0.76      0.79       279
       Happy       0.93      0.97      0.95       298
     Neutral       1.00      1.00      1.00       367
       Panic       0.90      0.82      0.86       274
     Sadness       0.81      0.91      0.86       284

    accuracy                           0.90      1502
   macro avg       0.89      0.89      0.89      1502
weighted avg       0.90      0.90      0.90      1502

[[211   6   0  22  40]
 [  5 289   0   1   3]
 [  0   0 367   0   0]
 [ 20  10   0 225  19]
 [ 18   5   0   2 259]]
